### Descargamos el contenido de los links en archivos de texto

In [1]:
import numpy as np
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
import urllib.request
import os
import re
from tqdm import tqdm
from random import sample

#### Habian archivos que tenian la info duplicada, asi que tenemos que poder borrar todo lo que sobra

In [2]:
def borrarRepetidos(path):
    file = open(path, 'r')
    texto = file.read()
    res = re.search('EOFNAME', texto)
    file.close()
    
    if res == None:
        return
    
    start,_ = res.span()
    texto = texto[:start]
    
    file = open(path, 'w')
    file.write(texto)
    file.close()
        

### Va a ser util poder conseguir cada campo del archivo

In [3]:
def get_file_field(texto, fieldRegexp):  
    res = re.search(fieldRegexp, texto)
    if res == None:
        return ""
    start, end = res.span()
    res = texto[end:].split("\n", 1)[0]
    if res[-1] == ' ':
        res = res[:-1]
    return res


In [4]:
regexType = r'EDGE_WEIGHT_TYPE(( |\t)+)?:(( |\t)+?)'
regexFormat =r'EDGE_WEIGHT_FORMAT(( |\t)+)?:(( |\t)+?)'
regexName =r'NAME(( |\t)+)?:(( |\t)+?)'

In [5]:
def bajar_archivos(file):
    f = open(file, 'r')
    lines=f.readlines()
    for index,line in enumerate(lines):
        print(index)
        response = urllib.request.urlopen(line)
        try:
            html = response.read()
        except:
            index=index-1
            continue
        soup = BeautifulSoup(html,"html5lib")
        text = soup.get_text(strip=True)
        text += "\n"
        name = get_file_field(text, regexName)
        path = "Data/"+name+".txt"
        f1 = open(path, "a")
        f1.write(text)
        f1.close()
        borrarRepetidos(path)

In [16]:
########bajar_archivos('links.txt')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110


### Separamos los archivos por campo

In [6]:
def split_by_type(path, _type, _format):
    path=str(path)
    files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]    
    for file in files:
        file=str(file)
        f = open(path+file, 'r')
        texto = f.read()
        fileType = get_file_field(texto, _type)
        typePath= path+fileType+'/'
        
        if not os.path.exists(typePath):
            os.mkdir(typePath)
        os.rename(path+file, typePath+'/'+file)
        
        isExplicit = fileType == "EXPLICIT"
        
        if isExplicit:
            fileFormat = get_file_field(texto, _format)
            formatPath = typePath + fileFormat + '/'
            if not os.path.exists(formatPath):
                os.mkdir(formatPath)
            os.rename(typePath+'/'+file, formatPath+'/'+file)    

In [28]:
split_by_type('Data/', regexType, regexFormat)

### Empezamos a pasar los txt a csv

#### Primero tenemos que copiar el arbol de carpetas para poder crear todos los csv ahi

In [7]:
def getFoldersTree(path):
    folders = [os.path.join(path, f) for f in os.listdir(path) if os.path.isdir(os.path.join(path, f))]
    if len(folders) == 0:
        return {}
    else:
        res = {}
        
        for folder in folders:
            key = folder[folder.rfind('/')+1:]
            res[key] = getFoldersTree(folder)
        
        return res

In [8]:
def createFoldersTree(path, tree):
    if len(tree) == 0:
        return
    for folder in tree:
        newDir = os.path.join(path, folder)
        os.mkdir(newDir)
        createFoldersTree(newDir, tree[folder])

In [9]:
def copyFoldersTree(src, dst):
    if not os.path.isdir(dst):
        os.mkdir(dst)
    tree = getFoldersTree(src)
    createFoldersTree(dst, tree)

In [10]:
txtPath = 'Data'
csvPath = 'DataCSV'
nueva = 'DataTesting'
nueva2 = 'DataValidacionOut'
nueva3 = 'DataTrainingOut'
nueva4 = 'DataMemoria'

In [60]:
copyFoldersTree(csvPath, nueva4)
# copyFoldersTree(csvPath, nueva3)

##### Empezamos por todos los que tengan distancia euclidia (Todos menos EXPLICIT)

#### Toma la distancia y lo pasa a matriz de adyacencia

In [6]:
def nint(x):
    if(x > 0):
        return (int)(x+0.5)
    return (int)(x-0.5)

def ATTDistance(p1, p2):
    deltaX = p1[0] - p2[0]
    deltaY = p1[1] - p2[1]
    Rij = np.sqrt( (deltaX*deltaX + deltaY*deltaY) / 10.0)
    Tij = nint(Rij)
    if(Tij < Rij):
        return Tij+1

    return Tij

def getGEOCoords(p):
    deg = nint(p[0])
    minimo = p[0] - deg
    latitud = np.pi * (deg + 5.0 * minimo / 3.0) / 180.0
    deg = nint(p[1])
    minimo = p[1] - deg
    longitud = np.pi * (deg + 5.0 * minimo / 3.0) / 180.0

    return np.array([latitud, longitud])


def GEODistance(p1, p2):
    coordsP1 = getGEOCoords(p1)
    coordsP2 = getGEOCoords(p2)

    R = 6378.388

    q1 = np.cos(coordsP1[1] - coordsP2[1])
    q2 = np.cos(coordsP1[0] - coordsP2[0])
    q3 = np.cos(coordsP1[0] + coordsP2[0])
    return nint(R * np.arccos( 0.5*((1.0+q1)*q2 - (1.0-q1)*q3))+ 1.0)

def EUC2DDistance(p1, p2):
    deltaX = p1[0] - p2[0]
    deltaY = p1[1] - p2[1]
    return nint( np.sqrt(deltaX*deltaX + deltaY*deltaY) + 0.5 )

def CEIL2DDistance(p1, p2):
    deltaX = p1[0] - p2[0]
    deltaY = p1[1] - p2[1]
    return np.ceil(np.sqrt(deltaX*deltaX + deltaY*deltaY) + 0.5 )

getDistance = {
    'ATT': ATTDistance,
    'CEIL_2D': CEIL2DDistance,
    'EUC_2D': EUC2DDistance,
    'GEO': GEODistance,    
}

In [9]:
def guardar_matriz(df, tipo, path):
    f = open(path, "w")
    f.write(str(len(csv))+',\n')
    N= len(df)
    
    for i in range(N):
        xA = float(df.iloc[i]['x'])
        yA = float(df.iloc[i]['y'])
        p1=np.array([xA, yA])
        for j in range(N):
            xB = float(df.iloc[j]['x'])
            yB = float(df.iloc[j]['y'])
            p2=np.array([xB, yB])
            f.write(str(getDistance[tipo](p1, p2))+',')
        
        if i < N-1:
            f.write('\n')
    f.close()

In [11]:
excluded = 'EXPLICIT'
folders = [os.path.join(txtPath, f) for f in os.listdir(txtPath) if f != excluded]

files = []
for path in folders:
    for file in os.listdir(path):
        if os.path.isfile(os.path.join(path, file)):
            files.append(os.path.join(path,file))
files

['Data/CEIL_2D/pla85900.txt',
 'Data/CEIL_2D/pla7397.txt',
 'Data/CEIL_2D/pla33810.txt',
 'Data/CEIL_2D/dsj1000.txt',
 'Data/GEO/gr666.txt',
 'Data/GEO/gr137.txt',
 'Data/GEO/gr431.txt',
 'Data/GEO/burma14.txt',
 'Data/GEO/gr96.txt',
 'Data/GEO/ulysses16.tsp.txt',
 'Data/GEO/gr202.txt',
 'Data/GEO/ulysses22.tsp.txt',
 'Data/GEO/gr229.txt',
 'Data/GEO/ali535.txt',
 'Data/EUC_2D/eil51.txt',
 'Data/EUC_2D/rl1889.txt',
 'Data/EUC_2D/pr226.txt',
 'Data/EUC_2D/vm1748.txt',
 'Data/EUC_2D/rat195.txt',
 'Data/EUC_2D/rl5934.txt',
 'Data/EUC_2D/fl1577.txt',
 'Data/EUC_2D/tsp225.txt',
 'Data/EUC_2D/nrw1379.txt',
 'Data/EUC_2D/pr439.txt',
 'Data/EUC_2D/pr124.txt',
 'Data/EUC_2D/usa13509.txt',
 'Data/EUC_2D/gil262.txt',
 'Data/EUC_2D/rat575.txt',
 'Data/EUC_2D/u2319.txt',
 'Data/EUC_2D/d2103.txt',
 'Data/EUC_2D/pr264.txt',
 'Data/EUC_2D/pcb3038.txt',
 'Data/EUC_2D/rd400.txt',
 'Data/EUC_2D/pr1002.txt',
 'Data/EUC_2D/berlin52.txt',
 'Data/EUC_2D/vm1084.txt',
 'Data/EUC_2D/fl3795.txt',
 'Data/EUC_2D/a

##### Ordeno los archivos de menor a mayor para poder hacer mas csvs en menos tiempo

In [12]:
def fileSize(file):
    csv= pd.read_csv(file,delim_whitespace=True, skiprows= [0,1,2,3,4,5], names=["Nodo", "x", "y"])
    return len(csv)-1

# files = np.array(files)

# largos = {}
# for file in files:
#     fSize = fileSize(file)
#     if fSize < 490:
#         largos[csvPath+file[file.find('/'):]] = fSize
        
# keys = list(largos)
# filesValidacion = sample(keys, int(len(keys)*0.3))
# filesTraining = []
# for key in largos:
#     if key in filesValidacion:
#         continue
#     filesTraining.append(key)

# print(filesValidacion)

NameError: name 'filesValidacion' is not defined

In [16]:
filesTesting = [] 
for file in files:
    N = fileSize(file)
    if 490 < N < 3000:
        filesTesting.append(file)

In [17]:
len(filesTesting)

30

In [18]:
for file in filesTesting:
    os.system(f'cp {file} {"DataTesting"+file[file.find("/"):]}')

In [53]:
for file in filesTraining:
    os.system(f'cp {file} {"DataTraining"+file[file.find("/"):]}')
    
for file in filesValidacion:
    os.system(f'cp {file} {"DataValidacion"+file[file.find("/"):]}')



In [5]:
import numpy as np

tito = np.array([15, 17, 23, 49, 52, 53, 71, 77, 77, 97, 100, 101, 101, 101, 101, 101, 101, 102, 106, 108, 125, 128, 131, 137, 138, 145, 151, 151, 151, 153, 160, 196, 199, 201, 201, 203, 226, 226, 227, 230, 263, 265, 281, 300, 318, 401, 418, 432, 440, 443, 494, 533, 536, 575, 576, 655, 658, 667, 725, 784, 1001, 1002, 1061, 1085, 1174, 1292, 1305, 1324, 1380, 1401, 1433, 1578, 1656, 1749, 1818, 1890, 2104, 2153, 2320, 2393, 3039, 3796, 4462, 5916, 5935, 7398, 11850, 13512, 14052, 18513, 33811, 85901])

menoresA500 = 0
grandesPeroUtiles = 0

for size in tito:
    if size < 490:
        menoresA500 += 1
    if size > 490 and size < 3000:
        grandesPeroUtiles += 1

print(menoresA500)
print(grandesPeroUtiles)


50
30


In [113]:
print(len(lens))
print(lens)

92
[15, 17, 23, 49, 52, 53, 71, 77, 77, 97, 100, 101, 101, 101, 101, 101, 101, 102, 106, 108, 125, 128, 131, 137, 138, 145, 151, 151, 151, 153, 160, 196, 199, 201, 201, 203, 226, 226, 227, 230, 263, 265, 281, 300, 318, 401, 418, 432, 440, 443, 494, 533, 536, 575, 576, 655, 658, 667, 725, 784, 1001, 1002, 1061, 1085, 1174, 1292, 1305, 1324, 1380, 1401, 1433, 1578, 1656, 1749, 1818, 1890, 2104, 2153, 2320, 2393, 3039, 3796, 4462, 5916, 5935, 7398, 11850, 13512, 14052, 18513, 33811, 85901]


In [ ]:
#Ultimo que estaba en progreso.
#Estoy con Data/EUC_2D/nrw1379.txt

In [13]:
for file in tqdm(files):
#Leo el csv y le saco el EOF si lo tiene
    print(f"Estoy con {file}\n")
    csv= pd.read_csv(file,delim_whitespace=True, skiprows= [0,1,2,3,4,5], names=["Nodo", "x", "y"])

    if len(csv) < 1379 or len(csv) > 20000 :
        continue
    if csv.iloc[-1]['Nodo'] == 'EOF':
        csv = csv[:-1]

    nuevoPath = csvPath+file[file.find('/'):file.rfind('.')]+'.csv'
    f = open(file)
    texto = f.read()
    tipo = get_file_field(texto, regexType)
    f.close()
    guardar_matriz(csv, tipo,nuevoPath)


 57%|█████▋    | 52/92 [00:00<00:00, 516.21it/s]

Estoy con Data/GEO/burma14.txt

Estoy con Data/GEO/ulysses16.tsp.txt

Estoy con Data/GEO/ulysses22.tsp.txt

Estoy con Data/ATT/att48.txt

Estoy con Data/EUC_2D/eil51.txt

Estoy con Data/EUC_2D/berlin52.txt

Estoy con Data/EUC_2D/st70.txt

Estoy con Data/EUC_2D/pr76.txt

Estoy con Data/EUC_2D/eil76.txt

Estoy con Data/GEO/gr96.txt

Estoy con Data/EUC_2D/rat99.txt

Estoy con Data/EUC_2D/kroC100.txt

Estoy con Data/EUC_2D/kroB100.txt

Estoy con Data/EUC_2D/kroD100.txt

Estoy con Data/EUC_2D/kroE100.txt

Estoy con Data/EUC_2D/kroA100.txt

Estoy con Data/EUC_2D/rd100.txt

Estoy con Data/EUC_2D/eil101.txt

Estoy con Data/EUC_2D/lin105.txt

Estoy con Data/EUC_2D/pr107.txt

Estoy con Data/EUC_2D/pr124.txt

Estoy con Data/EUC_2D/bier127.txt

Estoy con Data/EUC_2D/ch130.txt

Estoy con Data/EUC_2D/pr136.txt

Estoy con Data/GEO/gr137.txt

Estoy con Data/EUC_2D/pr144.txt

Estoy con Data/EUC_2D/ch150.txt

Estoy con Data/EUC_2D/kroA150.txt

Estoy con Data/EUC_2D/kroB150.txt

Estoy con Data/EUC_2D/pr1

 75%|███████▌  | 69/92 [06:28<02:47,  7.27s/it] 

Estoy con Data/EUC_2D/fl1400.txt



 76%|███████▌  | 70/92 [12:32<06:04, 16.57s/it]

...

Estoy con Data/EUC_2D/rl1889.txt



 83%|████████▎ | 76/92 [1:01:25<58:29, 219.34s/it]

Estoy con Data/EUC_2D/d2103.txt



 83%|████████▎ | 76/92 [1:02:51<13:13, 49.62s/it] 


KeyboardInterrupt: 